In [ ]:
# using property class
class Celsius:
    def __init__(self, temperature=0):
        self.temperature = temperature

    def to_fahrenheit(self):
        return (self.temperature * 1.8) + 32

    # getter
    @property
    def temperature(self):
        print("Getting value...")
        return self._temperature

    # setter

    @temperature.setter
    def temperature(self, value):
        print("Setting value...")
        if value < -273.15:
            raise ValueError("Temperature below -273.15 is not possible")
        self._temperature = value

    # creating a property object
    #temperature = property(get_temperature, set_temperature)


human = Celsius(-273)
human.temperature
#human.to_fahrenheit()

#human.temperature = -300

In [ ]:
import numpy as np

l = 100
eg = np.linspace(-7, 7, num=l)
#eg = np.random.shuffle(eg)

rng = np.random.default_rng()
rng.shuffle(eg)
#print(eg)
print(eg)

In [ ]:
import pandas as pd
from datetime import datetime

path = r'/Users/sr2/OneDrive - University College London/PhD/Research/Missions/SWARM/Non-Flight Data/Analysis/Apr-22/data/extraction/mag/MAG-data_April-16.csv'
df = pd.read_csv(path)

df = df.drop(columns=['nec_x','nec_y','nec_z'])

df=df.iloc[::20000]

def splitDatetime(df):
    #Splits datetime into date & utc, then reorders the df
    temp_df = df["datetime"].str.split(" ", n = 1, expand = True)
    df["date"] = temp_df [0]
    df["utc"] = temp_df [1]
    #df = df.reset_index().drop(columns=['datetime','index'])
    return df

df['alt'] = (df.pop('alt') / 1000 ) - 6371

def doy(x):
    day_of_year = x.timetuple().tm_yday
    return day_of_year
    
df = splitDatetime(df)
df['datetime'] = pd.to_datetime(df.pop('datetime'), format='%Y-%m-%d %H:%M:%S')
df['doy'] = df['datetime'].apply(doy)

def dns(x):
    dns = np.sin((2*np.pi*x)/365)
    return dns

def dnc(x):
    dnc = np.cos((2*np.pi*x)/365)
    return dnc

df['dns'] = df['doy'].apply(dns)
df['dnc'] = df['doy'].apply(dnc)

df.shape

In [ ]:
import requests
import xml.etree.ElementTree as ET
import datetime

def get_igrf(col):

    lat = col['lat']
    lon = col['long']
    alt = col['alt']
    date = col['date']
    
    path = 'https://geomag.bgs.ac.uk/web_service/GMModels/igrf/13/?latitude='+str(lat)+'&longitude='+str(lon)+'&altitude='+str(alt)+'&date='+str(date)+'&format=xml'
    
    response = requests.get(path)

    root = ET.fromstring(response.content)
    date = root[1].text

    for coors in root.findall('coordinates'):
        _lat = coors.find('latitude').text
        _lon = coors.find('longitude').text

    for fields in root.findall('field-value'):
        _ti = fields.find('total-intensity').text
        _dec = fields.find('declination').text
        _inc = fields.find('inclination').text
        _ni = fields.find('north-intensity').text
        _ei = fields.find('east-intensity').text
        _vi = fields.find('vertical-intensity').text
        _hi = fields.find('horizontal-intensity').text

    for secular in root.findall('secular-variation'):
        s_ti = secular.find('total-intensity').text
        s_dec = secular.find('declination').text
        s_inc = secular.find('inclination').text
        s_ni = secular.find('north-intensity').text
        s_ei = secular.find('east-intensity').text
        s_vi = secular.find('vertical-intensity').text
        s_hi = secular.find('horizontal-intensity').text

    #return pd.Series([_ti, _dec, _inc, _ni, _ei, _vi, _hi,
    #        s_ti, s_dec, s_inc, s_ni, s_ei, s_vi, s_hi])
    
    return pd.Series([_ti,_dec])
    #return _ti

df[['ti','dec']] = df.apply(get_igrf, axis=1)
print(df.head(5))

#print('Starting IGRF extraction at', pre_time)
#pre_time = datetime.datetime.now()
#df[['ti','dec','inc','ni','ei','vi','hi','s_ti','s_dec','s_inc','s_ni','s_ei','s_vi','s_hi']]= df.apply(get_igrf, axis=1)
#post_time = datetime.datetime.now()
#print('Finished IGRF extraction at', post_time)
#diff_time = post_time - pre_time
#print('Processing time:', diff_time)
#print(df.head())
#print(df.describe().transpose())



In [ ]:
from ftplib import FTP
import pandas as pd

ktp = FTP('ftp.gfz-potsdam.de')
ktp.login()
ktp.cwd('/pub/home/obs/Kp_ap_Ap_SN_F107')
years = [2014,2015,2016,2017,2018,2018,2019,2020,2021]
for i in years:
    filename = 'Kp_ap_'+str(i)+'.txt'
    with open(filename, 'wb' ) as file :
        ktp.retrbinary('RETR %s' % filename, file.write)

        file.close()
    print(f'Downloading data for {i}')

ktp.quit()

In [ ]:
from ftplib import FTP
import pandas as pd

ftp = FTP('ftp.gfz-potsdam.de')
ftp.login()
#ktp.cwd('/pub/home/obs/Kp_ap_Ap_SN_F107')
for i in years:
    filename = 'Kp_ap_'+str(i)+'.txt'
    with open(filename, 'wb' ) as file :
        ktp.retrbinary('RETR %s' % filename, file.write)

        file.close()
    print(f'Downloading data for {i}')

https://www.ngdc.noaa.gov

In [315]:
from pathlib import Path
import pandas as pd
import numpy as np
import datetime

path = Path(r'/Users/sr2/OneDrive - University College London/PhD/Research/'
        'Missions/SWARM/Non-Flight Data/Analysis/Mar-22/data/solar_max/')

dir_suffix = '2014'
#pre_classified = str(path) + '/' + dir_suffix +'-data-2022-03-03.csv'
post_classified = str(path) + '/ml_model/SG-filtered_14-15.csv'

def open_all(filename):
    print('Loading data...')
    df = pd.read_csv(filename)
    return df

df = open_all(post_classified)
#df = df.iloc[::10000]

Loading data...
